# Neural Network Implementation

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import lightgbm

from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold
from keras.layers import BatchNormalization, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import LeakyReLU

from IPython.display import display

from utils import *

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Load Data

In [ ]:
train = pd.read_csv('input/train.csv')
print(train.shape)

In [ ]:
test = pd.read_csv('input/test.csv')
print(test.shape)

## Preprocessing

Removing service columns

In [ ]:
targets = train[['TARGET']]
train.drop(columns=['SK_ID_CURR', 'TARGET'], inplace=True)

In [ ]:
test_ids = test['SK_ID_CURR']
test = test.drop(columns=['SK_ID_CURR'])

## Model

In [ ]:
model = Sequential()
model.add(Dense(24, activation='relu', input_shape=train.shape[1:]))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(24))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(24))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(1, activation='sigmoid'))
model.summary()